<a href="https://colab.research.google.com/github/nnilayy/Recurrent-Neural-Networks/blob/main/perfect_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import gc

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import roc_auc_score

import numpy as np
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

from keras.models import Sequential
from keras.layers import Dense
# measure roc auc score metric 
from tensorflow.keras.metrics import AUC

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_terms=pd.read_csv('/content/drive/MyDrive/CAFA_PROTEIN_FUNCTION/Train/train_terms.tsv',sep='\t')
# train_terms.head(50)

In [ ]:
train_terms.head()

,EntryID,term,aspect
0,A0A009IHW8,GO:0008152,BPO
1,A0A009IHW8,GO:0034655,BPO
2,A0A009IHW8,GO:0072523,BPO
3,A0A009IHW8,GO:0044270,BPO
4,A0A009IHW8,GO:0006753,BPO


In [ ]:
total=train_terms['term'].value_counts().sum()
classes=train_terms['term'].value_counts()
classes_percent=classes/total*100
classes_percent[:1000].sum()

77.28622077036644

In [ ]:
classes_percent=pd.DataFrame(classes_percent)
classes_percent

,term
GO:0005575,1.732184
GO:0008150,1.719097
GO:0110165,1.701870
GO:0003674,1.466052
GO:0005622,1.319665
...,...
GO:0031772,0.000019
GO:0042324,0.000019
GO:0031771,0.000019
GO:0051041,0.000019


In [ ]:
term_frequency=pd.DataFrame(train_terms['term'].value_counts())
term_frequency.reset_index(level=0, inplace=True)
term_frequency=term_frequency.rename(columns={'index':'Terms','term': 'Frequency'})
term_frequency=term_frequency[:1500]
term_frequency.head()

,Terms,Frequency
0,GO:0005575,92912
1,GO:0008150,92210
2,GO:0110165,91286
3,GO:0003674,78637
4,GO:0005622,70785


In [ ]:
train_terms=train_terms[train_terms['term'].isin(term_frequency['Terms'])]
train_terms=train_terms.reset_index(drop=True)
train_terms

,EntryID,term,aspect
0,A0A009IHW8,GO:0008152,BPO
1,A0A009IHW8,GO:0034655,BPO
2,A0A009IHW8,GO:0044270,BPO
3,A0A009IHW8,GO:0006753,BPO
4,A0A009IHW8,GO:0044237,BPO
...,...,...,...
4420732,X5L565,GO:0003674,MFO
4420733,X5L565,GO:0016491,MFO
4420734,X5M5N0,GO:0005515,MFO
4420735,X5M5N0,GO:0005488,MFO


In [ ]:
train_ids=np.load('/content/drive/MyDrive/t5embeds/train_ids.npy')
train_ids=pd.DataFrame(train_ids)
train_ids

,0
0,P20536
1,O73864
2,O95231
3,A0A0B4J1F4
4,P54366
...,...
142241,A0A286YAI0
142242,A0A1D5NUC4
142243,Q5RGB0
142244,A0A2R8QMZ5


In [ ]:
train_embeds=np.load('/content/drive/MyDrive/t5embeds/train_embeds.npy')
train_embeds=pd.DataFrame(train_embeds)
train_embeds['1024']=train_ids[0]
train_embeds=train_embeds.set_index('1024')
train_embeds = train_embeds.rename(index={'1024': 'EntryID'})
train_embeds=train_embeds.rename_axis('EntryID')
train_embeds=train_embeds.reset_index(level=0)
train_embeds.head()

,EntryID,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,P20536,0.049488,-0.032935,0.032473,-0.033723,-0.059505,0.075936,-0.045860,-0.104476,-0.072112,...,-0.046516,-0.028773,-0.021878,-0.097883,0.056475,0.080950,-0.020938,-0.043532,0.096463,0.073070
1,O73864,-0.044616,0.064925,-0.080263,-0.075338,-0.004731,0.025408,-0.024685,-0.016568,-0.038980,...,-0.040173,0.003925,0.006130,0.007362,-0.066848,0.106882,-0.030134,0.026724,0.027879,-0.048430
2,O95231,-0.020128,-0.049779,0.007894,-0.000829,-0.047737,0.086453,-0.038107,-0.036379,0.029611,...,0.029388,0.008456,0.000697,0.013502,-0.005968,-0.011571,0.005704,-0.036103,0.007693,0.106234
3,A0A0B4J1F4,-0.007515,0.060628,0.004045,0.027056,-0.021542,0.010380,-0.025064,-0.055834,0.068238,...,0.020792,0.023307,0.009009,0.018211,0.020820,-0.031738,0.013279,-0.018357,0.008087,0.010917
4,P54366,0.013468,0.041516,0.018435,-0.035595,0.008770,0.018699,-0.015452,-0.038092,-0.038326,...,-0.044742,-0.025432,-0.060886,-0.026915,0.026342,0.017237,0.014731,-0.016861,-0.016272,0.037054


In [ ]:
labels=train_terms.pivot(index ='EntryID', columns ='term',values='aspect')
labels.columns = [''.join(str(s).strip() for s in col if s) for col in labels.columns]
labels=labels.reset_index(level=0)
labels

<ipython-input-12-67af1669c4af>:1: PerformanceWarning: The following operation may generate 4475912636 cells in the resulting pandas object.
  labels=train_terms.pivot(index ='EntryID', columns ='term',values='aspect')


In [ ]:
labels= labels.iloc[:, 0:].notnull().astype("int")
labels=labels.drop(columns=['EntryID'])

train_embeds=train_embeds.sort_values('EntryID')
train_embeds=train_embeds.reset_index(drop=True)
train_embeds=train_embeds.drop(columns=['EntryID'])
train_embeds.head()

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0.067778,-0.046573,0.001871,-0.008532,0.003735,0.046137,-0.059283,-0.049906,-0.005312,0.008608,...,-0.040293,-0.013106,-0.049407,-0.100177,0.066018,0.009035,-0.003596,-0.024547,0.034698,-0.030786
1,-0.016303,-0.001599,0.003910,0.073261,0.012426,0.028077,-0.040412,-0.093539,-0.017918,0.025457,...,0.011927,-0.033233,-0.031254,-0.005067,0.014727,0.081792,0.017501,-0.033023,0.053113,0.029803
2,0.007994,0.087365,-0.001631,0.037468,0.017858,0.025504,-0.011703,-0.084401,-0.016387,-0.034820,...,0.004973,-0.049626,-0.027136,-0.037160,-0.006266,-0.039532,0.001768,0.004620,-0.004204,0.001841
3,0.002134,0.005902,0.064059,0.008222,-0.033212,-0.009141,-0.021793,-0.082338,-0.004026,-0.018951,...,-0.052513,-0.002440,-0.015824,-0.067800,0.041472,-0.050621,-0.025166,0.056364,0.097354,0.032667
4,0.048641,0.019926,0.107575,0.016752,-0.050425,-0.016805,-0.041421,-0.151901,0.007275,0.029193,...,-0.099127,-0.062734,0.002741,-0.102748,-0.000900,-0.046363,0.002974,-0.036701,0.102472,0.004925


In [ ]:
train_embeds.to_csv('/content/drive/MyDrive/CAFA_PROTEIN_FUNCTION/train_test_data/train_embeds.csv')
labels.to_csv('/content/drive/MyDrive/CAFA_PROTEIN_FUNCTION/train_test_data/labels.csv')

In [ ]:
import gc
gc.collect()

0

In [ ]:
train_embeds=pd.read_csv('/content/drive/MyDrive/CAFA_PROTEIN_FUNCTION/train_test_data/train_embeds.csv')
labels=pd.read_csv('/content/drive/MyDrive/CAFA_PROTEIN_FUNCTION/train_test_data/labels.csv')

In [ ]:
train_embeds=train_embeds.drop(columns=['Unnamed: 0'])
# train_embeds.head()

In [ ]:
labels=labels.drop(columns=['Unnamed: 0'])
# labels.head()

In [ ]:
test_ids=np.load('/content/drive/MyDrive/t5embeds/test_ids.npy')
test_ids=pd.DataFrame(test_ids)
# test_ids.head()

In [ ]:
test_embeds=np.load('/content/drive/MyDrive/t5embeds/train_embeds.npy')
test_embeds=pd.DataFrame(test_embeds)
# test_embeds.head()

In [ ]:
test_embeds=test_embeds.set_index(test_ids[0])
test_embeds = test_embeds.rename(index={'0': 'EntryID'})
test_embeds=test_embeds.rename_axis('EntryID')
test_embeds=test_embeds.reset_index(level=0)
# test_embeds.head()
# test_embeds.sort_values('EntryID')

ValueError: ignored

In [ ]:
test_embeds=test_embeds.sort_values('EntryID')
test_embeds=test_embeds.reset_index(drop=True)
# test_embeds.head()

In [ ]:
train_embeds=train_embeds.drop(columns=['EntryID'])
labels=labels.drop(columns=['EntryID'])

In [ ]:
selected_terms=set(labels.columns)
print(selected_terms)

In [ ]:
train_embeds=train_embeds.to_numpy()
labels=labels.to_numpy()

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(train_embeds, labels, shuffle=True, random_state=42)

In [ ]:
nfeats = x_train.shape[1]
nlabels = y_train.shape[1]
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=nfeats))
model.add(Dense(128, activation='relu'))
model.add(Dense(nlabels, activation='sigmoid'))
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=[AUC()])
model.fit(x_train, y_train, epochs=15, batch_size=128, validation_data=(x_valid, y_valid))

In [ ]:
model.save('model_ann.h5')

In [ ]:
from tensorflow.keras.saving import load_model
model=load_model('/kaggle/input/model-ann/model_ann.h5')

In [ ]:
y_hat = model.predict(x_valid)
scores = pd.DataFrame(columns=list(selected_terms), index=['roc_auc'])
for i, term in enumerate(selected_terms):
    score = roc_auc_score(y_valid[:, i], y_hat[:, i])
    scores[term] = score

scores.mean(axis=1)

In [ ]:
y_valid[:, i]

In [ ]:
test_ids = np.load('/kaggle/input/t5embeds/test_ids.npy')
x_test = np.load('/kaggle/input/t5embeds/test_embeds.npy')

In [ ]:
del x_train, y_train, x_valid, y_valid, labels
gc.collect()

In [ ]:
predictions = model.predict(x_test)
# del x_test
gc.collect()

chunk_size = 5_000
chunks = [range(i, min(i + chunk_size, len(predictions))) for i in range(0, len(predictions), chunk_size)]
final_sub = pd.DataFrame()  # Create an empty DataFrame to hold the final result
print(f"processing {len(chunks)} chunks of {chunk_size} predictions each")

for chunk in chunks:
    print(f"processing chunk {chunk}")
    sub = pd.DataFrame(data=predictions[chunk], columns=list(selected_terms), index=test_ids[chunk])
    sub = sub.T.unstack().reset_index(name='prediction')
    sub = sub.loc[sub['prediction'] > 0]
    final_sub = pd.concat([final_sub, sub])  # Concatenate current chunk DataFrame to the final DataFrame

final_sub.head()

In [ ]:
final_sub.to_csv('submission.tsv', sep='\t', index=False, header=False)

--------------------------------------------------------------------------------

--------------------------------------------------------------------------------

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/CAFA_PROTEIN_FUNCTION/IA.txt", sep="\t",header=None, names=['GO Terms','Predictions'])
df=pd.DataFrame(df)
df

,GO Terms,Predictions
0,GO:0000001,0.000000
1,GO:0000002,3.103836
2,GO:0000003,3.439404
3,GO:0000011,0.056584
4,GO:0000012,6.400377
...,...,...
43243,GO:2001083,7.159871
43244,GO:2001084,7.592457
43245,GO:2001085,7.159871
43246,GO:2001147,5.554589


In [ ]:
df[df['Predictions']>0]

,GO Terms,Predictions
1,GO:0000002,3.103836
2,GO:0000003,3.439404
3,GO:0000011,0.056584
4,GO:0000012,6.400377
5,GO:0000017,0.514573
...,...,...
43242,GO:2001082,7.159871
43243,GO:2001083,7.159871
43244,GO:2001084,7.592457
43245,GO:2001085,7.159871
